### Задание
Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"

Cохранить ее в pandas dataframe

Cохранить pandas dataframe в excel

Cохранить pandas dataframe в pickle

Cохранить pandas dataframe в БД

In [65]:
import requests
import pandas as pd
import time
import openpyxl

In [ ]:
objids = []
offset_ = 0
limit_ = 100

while len(objids) < 2000:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    try:
        res = requests.get(url)
        objects_data = res.json()

        objects_list = objects_data.get('data').get('list')
        objids.extend([x.get('objId') for x in objects_list])
    except:
        time.sleep(5)
        res = requests.get(url)
        objects_data = res.json()

        objects_list = objects_data.get('data').get('list')
        objids.append([x.get('objId') for x in objects_list])

    offset_ += 100

In [ ]:
objids[:10]

In [ ]:
all_res = []
for uid in objids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{uid}'
    try:
        res = requests.get(url)
        if res.ok:
            res = res.json()
            all_res.append(res.get("data"))
    except Exception:
        continue

all_res[0]

In [ ]:
len(all_res)

In [ ]:
houses = pd.DataFrame(data = pd.json_normalize(all_res))

In [ ]:
house_data = houses[[
            "region",
            "floorMax",
            "objElemLivingCnt",
            "objReady100PercDt",
            "objSquareLiving",
            "objElemParkingCnt",
            "objPriceAvg"
        ]]

In [ ]:
house_data.head()

In [ ]:
writer = pd.ExcelWriter('house_data.xlsx')
house_data.to_excel(writer)
writer.save()

In [ ]:
house_data.to_pickle('house_data.pickle')

In [ ]:
import sqlite3
con = sqlite3.connect("house_data.db")

In [ ]:
house_data.to_sql('house_data', con=con)

In [ ]:
n = 1
for uid in objids[:10]:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{uid}'
    img_data = requests.get(url)
    if img_data.ok:
        with open(f'image_{n}.jpg', 'wb+') as picture:
            picture.write(img_data.content)
    n += 1